# imports and data

In [1]:
data_dir = '/home/ubuntu/data/iceberg'

import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, Cropping2D
from keras.layers import MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.optimizers import Adam
import cv2
import keras
import os
from keras.callbacks import TensorBoard

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras import backend as K
from keras.utils.np_utils import to_categorical
from keras.layers.merge import Add, Concatenate

from PIL import Image

import keras
print(keras.__version__)

Using TensorFlow backend.


2.1.1


In [2]:
vgg_base = VGG16(weights='imagenet', include_top=False)

In [3]:
vgg_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [4]:
np.random.seed(1234)
from scipy.misc import imresize
from skimage.transform import resize

def get_scaled_imgs(df):
    imgs = []

    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)

#         band_1 = np.array(row['band_1']).reshape(75, 75)
#         band_2 = np.array(row['band_2']).reshape(75, 75)
#         band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
#         band_1 *= (255.0/band_1.max())
#         band_2 *= (255.0/band_2.max())
#         band_3 *= (255.0/band_3.max())

        img = resize(np.dstack((band_1, band_2, band_3)), (224, 224), mode='constant')
        imgs.append(img)

    return np.array(imgs)

def get_more_images(imgs):
    
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        c=imgs[i,:,:,2]
        
        av=cv2.flip(a,1)
        ah=cv2.flip(a,0)
        bv=cv2.flip(b,1)
        bh=cv2.flip(b,0)
        cv=cv2.flip(c,1)
        ch=cv2.flip(c,0)
        
        vert_flip_imgs.append(np.dstack((av, bv, cv)))
        hori_flip_imgs.append(np.dstack((ah, bh, ch)))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h))
    
    return more_images

In [5]:
df_train = pd.read_json(os.path.join(data_dir, 'train.json'))
Xtrain = get_scaled_imgs(df_train)
Ytrain = np.array(df_train['is_iceberg'])

df_train.inc_angle = df_train.inc_angle.replace('na',0)
idx_tr = np.where(df_train.inc_angle>0)

Ytrain = Ytrain[idx_tr[0]]
Xtrain = Xtrain[idx_tr[0],...]
Xinc = df_train.inc_angle[idx_tr[0]]


In [6]:
Xtrain.shape, Xinc.shape, Ytrain.shape

((1471, 224, 224, 3), (1471,), (1471,))

In [7]:
Xvalid = Xtrain[0:300]
Xinc_valid = Xinc[0:300]
Yvalid = Ytrain[0:300]

Xtrain_train = Xtrain[300:1471]
Xinc_train = Xinc[300:1471]
Ytrain_train = Ytrain[300:1471]

In [8]:
# remove all 3 below if you just want orginal data
Xtrain_train = get_more_images(Xtrain_train)
Xinc_train = np.concatenate((Xinc_train,Xinc_train,Xinc_train))
Ytrain_train = np.concatenate((Ytrain_train,Ytrain_train,Ytrain_train))

In [9]:
Xtrain_train.shape, Xinc_train.shape, Ytrain_train.shape

((3513, 224, 224, 3), (3513,), (3513,))

In [10]:
Xvalid.shape, Xinc_valid.shape, Yvalid.shape

((300, 224, 224, 3), (300,), (300,))

In [11]:
Yvalid[Yvalid == 0].shape, Yvalid[Yvalid == 1].shape

((144,), (156,))

In [12]:
def create_precomputed_data(model, data, data_labels):
    conv_features = model.predict(data)
    labels_onehot = to_categorical(data_labels)
    labels = data_labels
    return (conv_features, labels_onehot, labels)

In [13]:
trn_conv_features, trn_labels, trn_labels_1 = create_precomputed_data(vgg_base, Xtrain_train, Ytrain_train)
val_conv_features, val_labels, val_labels_1 = create_precomputed_data(vgg_base, Xvalid, Yvalid)
all_conv_features, all_labels, all_labels_1 = create_precomputed_data(vgg_base, Xtrain, Ytrain)

In [14]:
trn_conv_features.shape, trn_labels.shape, val_conv_features.shape, val_labels.shape

((3513, 7, 7, 512), (3513, 2), (300, 7, 7, 512), (300, 2))

In [15]:
assert trn_conv_features.shape == (3513, 7, 7, 512), "trn_conv_features not as expected"
assert trn_labels.shape == (3513, 2), "trn_labels not as expected"

assert val_conv_features.shape == (300, 7, 7, 512), "val_conv_features not as expected"
assert val_labels.shape == (300, 2), "val_labels not as expected"

In [16]:
classifier_input_shape = (7, 7, 512)
classifier_input = Input(shape=classifier_input_shape)

# deeper model with inc angle 

In [17]:
batch_size = 256
        
tbCallBack = TensorBoard(log_dir='/home/ubuntu/data/tensorboardlogs/', histogram_freq=0, write_graph=True, write_images=True)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, verbose=1, epsilon=1e-4, mode='min')

In [18]:
inc_angle_input = Input(shape=(1,))

nf = 128
p = 0.5

x = Conv2D(nf,(3,3), activation='relu', padding='same')(classifier_input)
x = BatchNormalization(axis=1)(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)
x = Dropout(p)(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)
x = Dropout(p)(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)

# x = MaxPooling2D((1,2))(x)
x = Dropout(p)(x)
x = Conv2D(2,(3,3), padding='same')(x)

# x = GlobalAveragePooling2D()(x)
# x = Activation('softmax')(x)


x = Flatten()(x)

# x = GlobalAveragePooling2D()(x)
m = Concatenate()([inc_angle_input, x])
m = Dense(512, activation='relu')(m)
m = Dense(256, activation='relu')(m)
out = Dense(2, activation='sigmoid')(m)
# out = Activation('softmax')(m)

optimizer = Adam(lr=0.001, decay=0.0)
model_inc_angle = Model(inputs=[classifier_input, inc_angle_input], outputs=out)
model_inc_angle.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
# model_inc_angle.fit([trn_conv_features, Xinc], trn_labels, batch_size=batch_size, epochs=50, verbose=1,shuffle=True,
#           callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack], validation_split=0.25)


model_inc_angle.fit([trn_conv_features, Xinc_train], trn_labels,
                                          batch_size=128, 
                                          epochs=50,
                                          validation_data=([val_conv_features, Xinc_valid], val_labels),
                                          shuffle=True, 
                                          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack])

Train on 3513 samples, validate on 300 samples
Epoch 1/50
3513/3513 [==============================] - 5s 1ms/step - loss: 0.0179 - acc: 0.9946 - val_loss: 0.6437 - val_acc: 0.8533
Epoch 2/50
3513/3513 [==============================] - 5s 1ms/step - loss: 0.0233 - acc: 0.9927 - val_loss: 0.6423 - val_acc: 0.8550
Epoch 3/50
3513/3513 [==============================] - 5s 1ms/step - loss: 0.0188 - acc: 0.9940 - val_loss: 0.6415 - val_acc: 0.8600
Epoch 4/50
3513/3513 [==============================] - 5s 1ms/step - loss: 0.0131 - acc: 0.9964 - val_loss: 0.6446 - val_acc: 0.8600
Epoch 5/50
3513/3513 [==============================] - 5s 1ms/step - loss: 0.0165 - acc: 0.9949 - val_loss: 0.6501 - val_acc: 0.8600
Epoch 6/50
3513/3513 [==============================] - 5s 1ms/step - loss: 0.0171 - acc: 0.9946 - val_loss: 0.6502 - val_acc: 0.8583
Epoch 7/50
3513/3513 [==============================] - 5s 1ms/step - loss: 0.0197 - acc: 0.9926 - val_loss: 0.6490 - val_acc: 0.8600
Epoch 8/50
3513

In [22]:
model_inc_angle.save_weights('/home/ubuntu/data/iceberg/results/weights/vgg_valid088.h5')

In [23]:
# model.load_weights(filepath = '.mdl_wts.hdf5')
score = model_inc_angle.evaluate([all_conv_features, Xinc], all_labels, verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

1471/1471 [==============================] - 2s 1ms/step
Train score: 0.136294496302
Train accuracy: 0.970768184908


In [24]:
# take out the generated images by filtering out the images without 4 dp in inc angle

data_dir = '/home/ubuntu/data/iceberg'

df_test = pd.read_json(os.path.join(data_dir, 'test.json'))
df_test.inc_angle = df_test.inc_angle.replace('na',0)
df_real = df_test.loc[df_test.inc_angle == round(df_test.inc_angle, 4)]
Xtest_real = (get_scaled_imgs(df_real))
Xtest_inc_real = df_real.inc_angle


In [25]:
Xtest_real.shape, Xtest_inc_real.shape

((3412, 224, 224, 3), (3412,))

In [26]:
test_conv_features = vgg_base.predict(Xtest_real)

In [27]:
pred_test = model_inc_angle.predict([test_conv_features, Xtest_inc_real])

In [60]:
pred_test

array([[  1.41173959e-01,   8.58255684e-01],
       [  1.00119389e-03,   9.99097586e-01],
       [  9.99995470e-01,   6.51482105e-06],
       ..., 
       [  1.00000000e+00,   3.74338356e-08],
       [  9.94567513e-01,   5.27053280e-03],
       [  1.04387966e-03,   9.99075294e-01]], dtype=float32)

In [61]:
idx_pred_1 = (np.where(pred_test[:,1]>0.95))
idx_pred_0 = (np.where(pred_test[:,1]<0.05))

In [62]:
idx_pred_1[0].shape, idx_pred_0[0].shape

((1730,), (1432,))

In [63]:
Xinc_ex_real = np.expand_dims(Xtest_inc_real, axis=1)
Xinc_ex_train = np.expand_dims(Xinc_train, axis=1)

In [64]:
Xtrain_pl = np.concatenate((Xtrain_train, Xtest_real[idx_pred_1[0],...], Xtest_real[idx_pred_0[0],...]))
Ytrain_pl = np.concatenate((Ytrain_train, np.ones(idx_pred_1[0].shape[0]), np.zeros(idx_pred_0[0].shape[0])))
Xinc_pl = np.concatenate((Xinc_ex_train, Xinc_ex_real[idx_pred_1[0],...], Xinc_ex_real[idx_pred_0[0],...]))

In [3]:
Xtrain_pl.shape, Ytrain_pl.shape, Xinc_pl.shape

NameError: name 'Xtrain_pl' is not defined

In [2]:
import numpy as np
np.squeeze(Xinc_pl).shape

NameError: name 'Xinc_pl' is not defined

In [66]:
pseudo_conv_features, pseudo_labels, pseudo_labels_1 = create_precomputed_data(vgg_base, Xtrain_pl, Ytrain_pl)

In [72]:
inc_angle_input = Input(shape=(1,))

nf = 128
p = 0.5

x = Conv2D(nf,(3,3), activation='relu', padding='same')(classifier_input)
x = BatchNormalization(axis=1)(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)
x = Dropout(p)(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)
x = Dropout(p)(x)
x = Conv2D(nf,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=1)(x)

# x = MaxPooling2D((1,2))(x)
x = Dropout(p)(x)
x = Conv2D(2,(3,3), padding='same')(x)

# x = GlobalAveragePooling2D()(x)
# x = Activation('softmax')(x)


x = Flatten()(x)

# x = GlobalAveragePooling2D()(x)
m = Concatenate()([inc_angle_input, x])
m = Dense(512, activation='relu')(m)
m = Dense(256, activation='relu')(m)
out = Dense(2, activation='sigmoid')(m)
# out = Activation('softmax')(m)

optimizer = Adam(lr=0.001, decay=0.0)
model_inc_angle2 = Model(inputs=[classifier_input, inc_angle_input], outputs=out)
model_inc_angle2.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [74]:
# model_inc_angle.fit([trn_conv_features, Xinc], trn_labels, batch_size=batch_size, epochs=50, verbose=1,shuffle=True,
#           callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack], validation_split=0.25)


model_inc_angle2.fit([pseudo_conv_features, Xinc_pl], pseudo_labels,
                                          batch_size=128, 
                                          epochs=50,
                                          validation_data=([val_conv_features, Xinc_valid], val_labels),
                                          shuffle=True, 
                                          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack])

Train on 6675 samples, validate on 300 samples
Epoch 1/50
6675/6675 [==============================] - 10s 1ms/step - loss: 0.0465 - acc: 0.9831 - val_loss: 0.5333 - val_acc: 0.8833
Epoch 2/50
6675/6675 [==============================] - 9s 1ms/step - loss: 0.0414 - acc: 0.9861 - val_loss: 0.5455 - val_acc: 0.8867
Epoch 3/50
6675/6675 [==============================] - 9s 1ms/step - loss: 0.0355 - acc: 0.9879 - val_loss: 0.5635 - val_acc: 0.8900
Epoch 4/50
6675/6675 [==============================] - 9s 1ms/step - loss: 0.0312 - acc: 0.9900 - val_loss: 0.5891 - val_acc: 0.8900
Epoch 5/50
6675/6675 [==============================] - 9s 1ms/step - loss: 0.0288 - acc: 0.9901 - val_loss: 0.6089 - val_acc: 0.8900
Epoch 6/50
6675/6675 [==============================] - 9s 1ms/step - loss: 0.0280 - acc: 0.9912 - val_loss: 0.6378 - val_acc: 0.8783
Epoch 7/50
6675/6675 [==============================] - 9s 1ms/step - loss: 0.0246 - acc: 0.9916 - val_loss: 0.6481 - val_acc: 0.8800
Epoch 8/50
667

In [75]:
model_inc_angle2.save_weights('/home/ubuntu/data/iceberg/results/weights/vgg_valid088_2.h5')

In [92]:
# this is the right way round
submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': pred_test[:,1].reshape((pred_test[:,1].shape[0]))})
print(submission.head(10))

submission.to_csv('cnn_train_' + str(np.around(score[1], decimals=2))  + '.csv', index=False)

         id    is_iceberg
0  5941774d  3.952795e-01
1  4023181e  9.995781e-01
2  b20200e4  9.970840e-01
3  e7f018bb  9.993762e-01
4  4371c8c3  9.994198e-01
5  a8d9b1fd  9.996321e-01
6  29e7727e  1.797852e-32
7  92a51ffb  9.995446e-01
8  c769ac97  1.552835e-15
9  aee0547d  1.769691e-29


In [93]:
os.getcwd()


'/home/ubuntu/git/learningWithKaggle/ice'

In [94]:
np.around(score[1])

1.0

In [32]:
Yvalid[Yvalid==0].shape, Yvalid[Yvalid==1].shape

((144,), (156,))

In [58]:
classifier_model_v3.fit(trn_conv_features, trn_labels,
                                          batch_size=32, 
                                          epochs=50,
                                          validation_data=(val_conv_features, val_labels),
                                          shuffle=True, 
                                          callbacks=[tbCallBack])

Train on 3713 samples, validate on 700 samples
Epoch 1/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.5313 - acc: 0.7498 - val_loss: 0.7826 - val_acc: 0.6957
Epoch 2/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.3282 - acc: 0.8529 - val_loss: 0.4505 - val_acc: 0.8071
Epoch 3/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.2591 - acc: 0.8942 - val_loss: 0.2983 - val_acc: 0.8800
Epoch 4/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.2205 - acc: 0.9076 - val_loss: 0.4919 - val_acc: 0.7986
Epoch 5/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.1512 - acc: 0.9405 - val_loss: 1.0548 - val_acc: 0.6986
Epoch 6/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.1279 - acc: 0.9534 - val_loss: 0.4099 - val_acc: 0.8771
Epoch 7/50
3713/3713 [==============================] - 11s 3ms/step - loss: 0.0948 - acc: 0.9623 - val_loss: 0.4143 - val_acc: 0.8600
Epoch 8/

In [ ]:
# create pngs of the training and validation data from above - training has aug already. 

In [ ]:
icebergs = train[ train.is_iceberg == 1 ]
ships = train[ train.is_iceberg == 0 ]